In [1]:
###version 0.2.1 reset allowed for I factors
import numpy as np
from MS_data_functions import *
import pandas as pd
plt.rcParams['image.cmap'] = 'afmhot'
#change to %matplotlib inline if you want the plots not in a separate window
%matplotlib qt

TypeError: unsupported operand type(s) for *: 'int' and 'NoneType'

In [2]:
#These are user variables [i.e. you should change them]
fpath = r'C:\Users\vgx18551\Documents\Data\Metal_oxides\MS_files\161007_SA01_LiNH2MnO2\161007_LiNH2MnO2_original.csv'
save_fpath = r'C:\Users\vgx18551\Documents\Data\Metal_oxides\MS_files\161007_SA01_LiNH2MnO2\161007_LiNH2MnO2_fitted.csv'
log_fpath = r'C:\Users\vgx18551\Documents\Data\Metal_oxides\MS_files\161007_SA01_LiNH2MnO2\161007_LiNH2MnO2.log'
MFC1_offset = 0
MFC2_offset = 0
MFM_offset = 0

In [3]:
#Reload MS data if it's already been fitted [you can then bypass the data extract and fitting code]
#fits = np.loadtxt(save_fpath, delimiter=',')
#fit_times, fit_params, T_ms = fits[:, 0], fits[:, 1:-1], fits[:, -1]
#labels = ['f$_{Ar}$', 'f$_{N_{2}}$', 'f$_{NH_{3}}$', 'f$_{H_{2}}$']

In [4]:
#Extract data and punt out times, amus, pressures and gas fraction arrays
data = pd.read_csv(fpath, sep=',', header=29, usecols=[0, 2, 3, 4])
times, amus, ps = extract_mshist_pd(data, sums=20)
fracs = get_gasfracs(ps)

In [5]:
contourf(times, amus, fracs, color_num=50)
#if you want to clip the z-data use:
#contourf(times, amus, np.clip(fracs, 0, 0.5))
#if you want to plot on a log scale:
contourf(times, amus, np.log(np.clip(fracs, 1e-5, 1)), zlabel='log(Gas Fraction)')
#if you want to plot on a square root scale:
#contourf(times, amus, np.sqrt(np.clip(fracs, 1e-5, 1)), zlabel=r'$\sqrt{Gas Fraction}$')
#if you want to only plot between certain times:
#t0, t1 = 10, 100 #in minutes
#i0, i1 = [np.searchsorted(times[0], t) for t in [t0, t1]]
#contourf(times[:, i0:i1], amus[:, i0:i1], fracs[:, i0:i1])

In [6]:
###Here's an example of how to self calibrate for pure NH3 running between 1200 and 1240 minutes###
###You must re-run the fit_MS_data() function afterwards in order to implement the changes###
all_sfs.self_calibrate('NH3', times, amus, fracs, x_range=[1200, 1240])
###If you ever want to go back to the initial values, then use the following:###
#all_sfs.reset()

In [7]:
#fit data here
fit_times, fit_params = fit_MS_data(times, fracs, time_range=None)
labels = ['f$_{Ar}$', 'f$_{N_{2}}$', 'f$_{NH_{3}}$', 'f$_{H_{2}}$']

In [16]:
#plot data here
mspec_plot(fit_times, fit_params, labels)

In [8]:
#load log data (print log_data[0] for the column names, and log_data = log_data.values to
#get numpy array of log_data
log_data = get_log_data(log_fpath)
#now get the temperatures in same array shape as the MS data
T_ms = get_T_ms(log_data, fit_times)

In [9]:
#plot MS data versus temperature
mspec_plot(T_ms, fit_params, labels)

In [12]:
#save MS data here
fits = np.column_stack((fit_times, fit_params, T_ms))
np.savetxt(save_fpath, fits, delimiter=',')